In [ ]:
#!pip install tqdm

In [1]:
import numpy as np
import torch

package_used = 'torch'

def uran_numpy(n):
    return np.random.rand(n)

def uran_torch(n):
    return torch.rand(n)

def gran_numpy(n, m):
    return np.random.randn(n, m)

def gran_torch(n, m):
    return torch.randn(n, m)
'''
def clp_numpy(n, B, x):
    u_hat = np.zeros(n)
    C = np.inf
    i = n + 1
    d = np.full(n, n)
    lamb = np.zeros(n + 1)
    F = np.zeros((n, n))
    F[-1, :] = x
    u = np.zeros(n)
    p = np.zeros(n)
    delta = np.zeros(n)
    while True:
        while True:
            if i != 1:
                i -= 1
                for j in range(d[i - 1], i + 1, -1):
                    F[j - 2, i - 1] = F[j - 1, i - 1] - u[j - 1] * B[j - 1, i - 1]
                p[i - 1] = F[i - 1, i - 1] / B[i - 1, i - 1]
                u[i - 1] = np.round(p[i - 1])
                y = (p[i - 1] - u[i - 1]) * B[i - 1, i - 1]
                delta[i - 1] = np.sign(y)
                lamb[i - 1] = lamb[i] + y * y
            else:
                u_hat = u
                C = lamb[0]
            if lamb[i - 1] >= C:
                break
        m = i
        while True:
            if i == n:
                return u_hat
            i += 1
            u[i - 1] = u[i - 1] + delta[i - 1]
            delta[i - 1] = -delta[i - 1] - np.sign(delta[i - 1])
            y = (p[i - 1] - u[i - 1]) * B[i - 1, i - 1]
            lamb[i - 1] = lamb[i] + y * y
            if lamb[i - 1] < C:
                break
        #for j in range(m, i - 1):
        #    d[j - 1] = i
        d[m - 1: i - 1] = i
        for j in range(m - 1, 0, -1):
            if d[j - 1] < i:
                d[j - 1] = i
            else:
                break

def clp_torch(n, B, x):
    u_hat = torch.zeros(n, dtype=B.dtype, device=B.device)
    C = torch.tensor(np.inf, dtype=B.dtype, device=B.device)
    i = n + 1
    d = torch.full((n,), n, dtype=torch.long, device=B.device)
    lamb = torch.zeros(n + 1, dtype=B.dtype, device=B.device)
    F = torch.zeros((n, n), dtype=B.dtype, device=B.device)
    F[-1, :] = x
    u = torch.zeros(n, dtype=B.dtype, device=B.device)
    p = torch.zeros(n, dtype=B.dtype, device=B.device)
    delta = torch.zeros(n, dtype=B.dtype, device=B.device)
    while True:
        while True:
            if i != 1:
                i -= 1
                for j in range(d[i - 1], i + 1, -1):
                    F[j - 2, i - 1] = F[j - 1, i - 1] - u[j - 1] * B[j - 1, i - 1]
                p[i - 1] = F[i - 1, i - 1] / B[i - 1, i - 1]
                u[i - 1] = torch.round(p[i - 1])
                y = (p[i - 1] - u[i - 1]) * B[i - 1, i - 1]
                delta[i - 1] = torch.sign(y)
                lamb[i - 1] = lamb[i] + y * y
            else:
                u_hat = u
                C = lamb[0]
            if lamb[i - 1] >= C:
                break
        m = i
        while True:
            if i == n:
                return u_hat
            i += 1
            u[i - 1] = u[i - 1] + delta[i - 1]
            delta[i - 1] = -delta[i - 1] - torch.sign(delta[i - 1])
            y = (p[i - 1] - u[i - 1]) * B[i - 1, i - 1]
            lamb[i - 1] = lamb[i] + y * y
            if lamb[i - 1] < C:
                break
        d[m - 1: i - 1] = i
        for j in range(m - 1, 0, -1):
            if d[j - 1] < i:
                d[j - 1] = i
            else:
                break
'''
def initial_gso_numpy(basis):
    """
    Compute the initial Gram–Schmidt decomposition of a row-based 'basis'.

    Returns:
      b_star : (n, dim) array of the orthogonal vectors b_i^*
      mu     : (n, n)   array, mu[i,j] = coefficient of b_j^* in b_i, for j < i
      B      : (n,)     array, B[j] = ||b_j^*||^2
    """
    n, dim = basis.shape
    b_star = np.zeros((n, dim), dtype=basis.dtype)
    mu     = np.zeros((n, n),    dtype=basis.dtype)
    B      = np.zeros(n,         dtype=basis.dtype)  # squared norms of b_j^*

    for i in range(n):
        # Start from b_i
        b_star[i] = basis[i].copy()
        for j in range(i):
            # Project b_i onto b_j^*
            mu[i, j] = np.dot(basis[i], b_star[j]) / B[j] if B[j] != 0 else 0
            b_star[i] -= mu[i, j] * b_star[j]
        B[i] = np.dot(b_star[i], b_star[i])

    return b_star, mu, B

def size_reduce_numpy(basis, b_star, mu, B, k, j):
    """
    Size-reduce row k with respect to row j using partial updates.

    Parameters
    ----------
    basis  : (n, dim)  row-based array of the basis
    b_star : (n, dim)  row-based array of the orthogonal vectors
    mu     : (n, n)    GSO coefficients
    B      : (n,)      squared norms of b_star
    k      : int       index of row being reduced
    j      : int       index of row used for reduction
    """
    q = np.round(mu[k, j])
    if np.isclose(q, 0.0):
        return  # no reduction needed

    # 1. Update b_k by subtracting q * b_j
    basis[k] -= q * basis[j]

    # 2. Update the GSO coefficients mu[k, t] for t in [0..j-1]
    for t in range(j):
        mu[k, t] -= q * mu[j, t]

    # 3. Update mu[k, j] by subtracting q
    mu[k, j] -= q

    # 4. Update b_k^*
    b_star[k] -= q * b_star[j]
    # 5. Update the squared norm B[k]
    B[k] = np.dot(b_star[k], b_star[k])

def update_gso_after_swap_numpy(basis, b_star, mu, B, k):
    """
    After swapping rows k and k-1 in 'basis', we do a *partial* fix-up:
      - Swap b_star[k] and b_star[k-1]
      - Swap mu-coefficients with indices (k, :) and (k-1, :)
      - Recompute mu[k,k-1], b_star[k], B[k], and possibly some mu[*][k].
    """
    # Swap the orthogonal vectors b_star[k] and b_star[k-1]
    temp = b_star[k].copy()
    b_star[k] = b_star[k-1]
    b_star[k-1] = temp

    # Swap the squared norms as well
    B[k], B[k-1] = B[k-1], B[k]

    # Swap all mu[k, t] <--> mu[k-1, t], for t < k-1
    for t in range(k-1):
        mu[k, t], mu[k-1, t] = mu[k-1, t], mu[k, t]

    # Now we must *recompute* mu[k, k-1], b_star[k], and B[k].
    # Because b_k changed to what b_{k-1} was, and vice versa.

    # Recompute mu[k,k-1] = (b_k dot b_{k-1}^*) / ||b_{k-1}^*||^2
    mu[k, k-1] = np.dot(basis[k], b_star[k-1]) / B[k-1] if B[k-1] != 0 else 0.0

    # Recompute b_star[k] = b_k - sum_{j=0}^{k-1} mu[k,j]*b_star[j]
    # but the partial trick is to only fix b_star[k] now that we've updated mu[k,k-1].

    # 1) Start fresh:
    b_star[k] = basis[k].copy()
    for j in range(k):
        b_star[k] -= mu[k, j] * b_star[j]

    # 2) Update B[k]
    B[k] = np.dot(b_star[k], b_star[k])

def lll_partial_update_numpy(basis, delta=0.75):
    """
    LLL reduction with partial GSO updates.
    Each row of 'basis' is a vector in R^dim.

    Parameters
    ----------
    basis : (n, dim) array
      Row-based representation of the lattice basis.
    delta : float
      Lovasz condition parameter in (0.25, 1).

    Returns
    -------
    basis : (n, dim) array
      LLL-reduced basis (modified in place and also returned).
    """
    # Convert to floating type for dot products etc.
    # basis = basis.astype(float)
    n, dim = basis.shape

    # -- 1) Compute initial GSO:
    b_star, mu, B = initial_gso_numpy(basis)

    # -- 2) LLL main loop:
    k = 1
    while k < n:
        # (a) Size Reduction: reduce b_k w.r.t. b_j for j = k-1..0
        for j in range(k-1, -1, -1):
            size_reduce_numpy(basis, b_star, mu, B, k, j)

        # (b) Check Lovász condition if k >= 1
        #     i.e.,  ||b_k^*||^2 >= (delta - mu[k,k-1]^2)*||b_{k-1}^*||^2
        if k > 0:
            lhs = B[k]
            rhs = (delta - mu[k, k-1]**2) * B[k-1]
            if lhs < rhs:
                # Swap rows k and k-1 in 'basis'
                basis[[k, k-1]] = basis[[k-1, k]]

                # Partial GSO fix-up after the swap
                update_gso_after_swap_numpy(basis, b_star, mu, B, k)

                # Step back
                k = max(k-1, 1)
                continue

        # Move forward
        k += 1

    return basis

def initial_gso_torch(basis: torch.Tensor):
    """
    Compute the initial Gram–Schmidt decomposition of a row-based 'basis'.

    Parameters
    ----------
    basis : (n, dim) torch.Tensor
      Each row of 'basis' is a vector in R^dim.

    Returns
    -------
    b_star : (n, dim) torch.Tensor
      Orthogonalized vectors b_i^* for each row b_i.
    mu     : (n, n) torch.Tensor
      mu[i, j] = (b_i dot b_j^*) / ||b_j^*||^2 for j < i, else 0
    B      : (n,) torch.Tensor
      B[j] = ||b_j^*||^2 (the squared norm of b_j^*)
    """
    n, dim = basis.shape

    b_star = torch.zeros_like(basis)
    mu = torch.zeros(n, n, dtype=basis.dtype, device=basis.device)
    B  = torch.zeros(n,    dtype=basis.dtype, device=basis.device)

    for i in range(n):
        # Start with b_i:
        b_star[i] = basis[i].clone()
        for j in range(i):
            # Project b_i onto b_j^*
            # mu[i,j] = (b_i dot b_j^*) / ||b_j^*||^2
            denom = B[j]
            if denom.abs() > 1e-15:  # avoid division by zero
                mu[i, j] = torch.dot(basis[i], b_star[j]) / denom
            else:
                mu[i, j] = torch.tensor(0.0, device=basis.device)

            # Subtract this projection from b_star[i]
            b_star[i] -= mu[i, j] * b_star[j]

        # B[i] = ||b_star[i]||^2
        B[i] = torch.dot(b_star[i], b_star[i])

    return b_star, mu, B

def size_reduce_torch(basis, b_star, mu, B, k, j):
    """
    Size-reduce row k with respect to row j using partial updates in PyTorch.

    Parameters
    ----------
    basis  : (n, dim) torch.Tensor
      Row-based array of the basis vectors.
    b_star : (n, dim) torch.Tensor
      Row-based array of the orthogonal vectors.
    mu     : (n, n)   torch.Tensor
      GSO coefficients.
    B      : (n,)     torch.Tensor
      Squared norms of b_star.
    k      : int
      Index of row being reduced.
    j      : int
      Index of row used for reduction.
    """
    q = torch.round(mu[k, j])
    # We'll treat "close to zero" as zero
    if torch.abs(q) < 1e-15:
        return  # no reduction needed

    # 1) Update b_k: b_k = b_k - q * b_j
    basis[k] -= q * basis[j]

    # 2) Update the GSO coefficients mu[k, t] for t in [0..j-1]
    #    mu[k, t] = mu[k, t] - q * mu[j, t]
    mu[k, :j] -= q * mu[j, :j]

    # 3) Update mu[k, j] by subtracting q
    mu[k, j] -= q

    # 4) Update b_k^*
    b_star[k] -= q * b_star[j]

    # 5) Update the squared norm B[k]
    B[k] = torch.dot(b_star[k], b_star[k])

def update_gso_after_swap_torch(basis, b_star, mu, B, k):
    """
    After swapping rows k and k-1 in 'basis', do a partial fix-up of the GSO data.

    Parameters
    ----------
    basis  : (n, dim) torch.Tensor
      Row-based basis matrix.
    b_star : (n, dim) torch.Tensor
      Orthogonal vectors.
    mu     : (n, n)   torch.Tensor
      GSO coefficients.
    B      : (n,)     torch.Tensor
      Squared norms of b_star.
    k      : int
      The row index that was swapped with (k-1).
    """
    # 1) Swap b_star[k] and b_star[k-1]
    temp = b_star[k].clone()
    b_star[k] = b_star[k-1]
    b_star[k-1] = temp

    # 2) Swap B[k] and B[k-1]
    tempB = B[k].clone()
    B[k]   = B[k-1]
    B[k-1] = tempB

    # 3) Swap mu[k,:] and mu[k-1,:], but only for columns < k-1
    #    i.e., mu[k, t] <--> mu[k-1, t]
    #    for t in [0..k-2]
    tmp_slice = mu[k, :k-1].clone()
    mu[k, :k-1] = mu[k-1, :k-1]
    mu[k-1, :k-1] = tmp_slice

    # 4) Recompute mu[k,k-1], b_star[k], and B[k].
    denom = B[k-1]
    if denom.abs() > 1e-15:
        mu[k, k-1] = torch.dot(basis[k], b_star[k-1]) / denom
    else:
        mu[k, k-1] = torch.tensor(0.0, device=basis.device)

    # 5) Recompute b_star[k] = b_k - sum_{j=0}^{k} mu[k,j] * b_star[j]
    b_star[k] = basis[k].clone()
    for j in range(k):
        b_star[k] -= mu[k, j] * b_star[j]

    # 6) Update B[k] = ||b_star[k]||^2
    B[k] = torch.dot(b_star[k], b_star[k])

def lll_partial_update_torch(basis, delta=0.75):
    """
    LLL reduction with partial GSO updates (PyTorch version).
    Each row of 'basis' is a vector in R^dim.

    Parameters
    ----------
    basis : (n, dim) torch.Tensor
      Row-based representation of the lattice basis.
    delta : float
      Lovasz condition parameter in (0.25, 1).

    Returns
    -------
    basis : (n, dim) torch.Tensor
      LLL-reduced basis (modified in-place and also returned).
    """
    # Make sure it's float (or double)
    # basis = basis.to(dtype=torch.double)  # double precision for safety

    # -- 1) Initial GSO
    b_star, mu, B = initial_gso_torch(basis)

    n = basis.shape[0]
    k = 1
    while k < n:
        # -- (a) Size Reduction: reduce b_k w.r.t. b_j for j = k-1..0
        for j in range(k-1, -1, -1):
            size_reduce_torch(basis, b_star, mu, B, k, j)

        # -- (b) Check Lovasz condition if k >= 1
        #     ||b_k^*||^2 >= (delta - mu[k,k-1]^2) * ||b_{k-1}^*||^2
        if k > 0:
            lhs = B[k]
            rhs = (delta - mu[k, k-1]**2) * B[k-1]

            if lhs < rhs:
                # Swap rows k and k-1 in 'basis'
                tmp_row = basis[k].clone()
                basis[k] = basis[k-1]
                basis[k-1] = tmp_row

                # Partial GSO fix-up
                update_gso_after_swap_torch(basis, b_star, mu, B, k)

                # Step back
                k = max(k-1, 1)
                continue

        k += 1

    return basis

def orth_numpy(B):
    A = B @ B.T
    return np.linalg.cholesky(A)

def orth_torch(B):
    A = B @ B.T
    return torch.linalg.cholesky(A).type(B.dtype)

def uran(n):
    if package_used == 'numpy':
        return uran_numpy(n)
    else:
        return uran_torch(n)

def gran(n, m):
    if package_used == 'numpy':
        return gran_numpy(n, m)
    else:
        return gran_torch(n, m)
'''
def clp(n, B, x):
    if package_used == 'numpy':
        return clp_numpy(n, B, x)
    else:
        return clp_torch(n, B, x)
'''
def red(B):
    if package_used == 'numpy':
        return lll_partial_update_numpy(B)
    else:
        return lll_partial_update_torch(B)

def orth(B):
    if package_used == 'numpy':
        return orth_numpy(B)
    else:
        return orth_torch(B)

# 新段落

In [2]:
import time
def CLP(B,r):
    n=r.shape[0]
    c=1e12
    i=n+1
    d=n*np.ones((n+1),dtype=np.int32)
    lmda=np.zeros((n+2),dtype=np.float32)
    F=np.zeros((n+1,n+1),dtype=np.float32)
    F[n,1:]=r.copy()
    u=np.zeros((n+1),dtype=np.int32)
    u_ans=np.zeros((n+1),dtype=np.int32)
    det=np.zeros((n+1),dtype=np.float32)
    p=np.zeros((n+1),dtype=np.float32)
    start_time = time.time()
    while True:
        cur_time = time.time()
        if cur_time - start_time > 10:
            return u_ans[1:]
        while(lmda[i]<c):
            if i!=1:
                i=i-1
                for j in range(d[i],i,-1):
                    F[j-1,i]=F[j,i]-u[j]*B[j-1,i-1]
                p[i]=F[i,i]/B[i-1,i-1]
                u[i]=round(p[i])
                y=(p[i]-u[i])*B[i-1,i-1]
                det[i]=np.sign(y)
                lmda[i]=lmda[i+1]+y**2
            else:
                u_ans=u.copy()
                c=lmda[1].copy()
        m=i
        while(lmda[i]>=c):
            if i==n:
                return u_ans[1:]
            else:
                i=i+1
                u[i]+=det[i]
                det[i]=-det[i]-np.sign(det[i])
                y=(p[i]-u[i])*B[i-1,i-1]
                lmda[i]=lmda[i+1]+y**2
        for j in range(m,i):
            d[j]=i
        for j in range(1,m):
            if d[j]<i:
                d[j]=i
            else:
                break

import torch

def CLP_torch(B, r):
    n = r.shape[0]
    c = torch.tensor(np.inf, device=B.device)
    i = n + 1
    d = n * torch.ones(n + 1, dtype=torch.int32, device=B.device)
    lmda = torch.zeros(n + 2, dtype=torch.float32, device=B.device)
    F = torch.zeros(n + 1, n + 1, dtype=torch.float32, device=B.device)
    F[n, 1:] = r.clone()
    u = torch.zeros(n + 1, dtype=torch.int32, device=B.device)
    u_ans = torch.zeros(n + 1, dtype=torch.int32, device=B.device)
    det = torch.zeros(n + 1, dtype=torch.int32, device=B.device)
    p = torch.zeros(n + 1, dtype=torch.float32, device=B.device)
    start_time = time.time()
    while True:
        cur_time = time.time()
        if cur_time - start_time > 10:
            return u_ans[1:]
        while True:
            if i != 1:
                i = i - 1
                for j in range(d[i], i, -1):
                    F[j - 1, i] = F[j, i] - u[j] * B[j - 1, i - 1]
                p[i] = F[i, i] / B[i - 1, i - 1]
                u[i] = torch.round(p[i]).int()
                y = (p[i] - u[i]) * B[i - 1, i - 1]
                det[i] = torch.sign(y)
                lmda[i] = lmda[i + 1] + y**2
            else:
                u_ans = u.clone()
                c = lmda[1].clone()
            if lmda[i] >= c:
                break
        m = i
        while True:
            if i == n:
                return u_ans[1:]
            else:
                i = i + 1
                u[i] += det[i]
                det[i] = -det[i] - torch.sign(det[i])
                y = (p[i] - u[i]) * B[i - 1, i - 1]
                lmda[i] = lmda[i + 1] + y**2
            if lmda[i] < c:
                break
        for j in range(m, i):
            d[j] = i
        for j in range(1, m):
            if d[j] < i:
                d[j] = i
            else:
                break

def clp(n, B, x):
    if package_used == 'numpy':
        return CLP(B, x)
    else:
        return CLP_torch(B, x)

In [3]:
import torch
torch.cuda.is_available()
#from google.colab import drive
from pathlib import Path
#drive.mount('/content/gdrive')
#model_path = '/content/gdrive/MyDrive/ML/Models/'
model_path = "./Models/"
model_path = Path(model_path)
if not model_path.exists():
    model_path.mkdir(parents=True)
    print(f'Created {model_path}')

Created Models


In [7]:
import numpy as np
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import time

#from utils import clp, red, orth
#import utils

#utils.package_used = 'torch'
package_used='torch'
class Config:
    def __init__(self):
        self.n = 10
        self.d = 2
        self.k = 2
        self.num_epochs = 1000000
        self.red_epoch_interval = 100
        self.batch_size = 1
        self.val_size = 16
        self.lr = 0.001
        self.seed = None
        self.device = 'cpu'#torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.dtype = torch.float32
        self.reduce_B = True
        self.save_interval = 10000

def loss_fn(n, V, e):
    return 1 / n * (V ** (-2 / n)) * torch.norm(e, p=2) ** 2

class ILC(nn.Module):
    def __init__(self, config):
        super(ILC, self).__init__()
        self.config = config
        self.n = config.n
        self.B = nn.Parameter(torch.randn(self.n, self.n, dtype=config.dtype, device=config.device))
        self.B.data = self.reduce_B(self.B.data)
        self.B.data = torch.diag(torch.ones(config.n, dtype=config.dtype, device=config.device))
        self.mask = self.generate_mask(self.n)

    def calculate_V(self, B):
        return torch.prod(torch.diag(B))

    def reduce_B(self, B):
        with torch.no_grad():
            B = orth(red(B))
            V = self.calculate_V(B)
            B = V ** (-1 / self.n) * B
        return B

    def generate_mask(self, n):
        # Keep B as a lower triangular matrix
        mask = torch.tril(torch.ones(n, n, dtype=torch.bool, device=self.config.device))
        return mask

    def mask_B(self, B):
        with torch.no_grad():
            B = B * self.mask
        return B

    def forward(self, y):
        e = y @ self.B
        V = self.calculate_V(self.B)
        loss = loss_fn(self.n, V, e)
        return loss

def calculate_y(n, z, B):
    with torch.no_grad():
        z = z.to(B.device)
        B = B.to(B.device)
        return z - clp(n, B, z @ B)

def iterative_lattice_construction(n):
    config = Config()
    config.n = n
    model = ILC(config)
    model.to(config.device)
    optimizer = optim.AdamW(model.parameters(), lr=config.lr)
    scheduler = StepLR(optimizer, step_size=10000, gamma=0.9)
    if config.seed is not None:
        torch.manual_seed(config.seed)
    else:
        torch.seed()
    last_total_loss = 0
    for epoch in tqdm(range(config.num_epochs)):
        loss = torch.tensor(0, dtype=config.dtype, device=config.device)
        z = torch.rand(config.batch_size, config.n, dtype=config.dtype, device=config.device)
        for i in range(config.batch_size):
            y = calculate_y(config.n, z[i], model.B.data)
            loss += model(y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        model.B.data = model.mask_B(model.B.data)
        #last_total_loss += loss.item() / config.batch_size
        #print('Epoch: {}, Loss: {}'.format(epoch, loss.item() / config.batch_size))
        if epoch % config.red_epoch_interval == config.red_epoch_interval - 1 and config.reduce_B:
            model.B.data = model.reduce_B(model.B.data)
            #with torch.no_grad():
            #  loss = torch.tensor(0, dtype=config.dtype, device=config.device)
            #  z = torch.rand(config.val_size, config.n, dtype=config.dtype, device=config.device)
            #  for i in range(config.val_size):
            #    y = calculate_y(config.n, z[i], model.B.data)
            #    loss += model(y)
            #print()
            #print(f'Epoch {epoch}, Loss: {loss.item() / config.batch_size}')
            last_total_loss = 0

            #print('Reduced B')
        if epoch % config.save_interval == config.save_interval - 1:
            print()
            #print(f'Epoch {epoch}, Loss: {loss.item()}')
            last_total_loss = 0
            torch.save(model.B, model_path / f'B_{model.config.n}_{epoch}.pt')
            print(f'Saved B_{model.config.n}_{epoch}.pt')
    return model


In [8]:
# Train the model: run this cell
if __name__ == '__main__':
    model = iterative_lattice_construction(10)
    print(model.B)
    torch.save(model, model_path / f'B_{model.config.n}_{time.time()}.pt')

  0%|          | 0/1000000 [00:00<?, ?it/s]

  1%|          | 9703/1000000 [02:40<4:32:42, 60.52it/s]


KeyboardInterrupt: 

In [52]:
config = Config()
config.n = 10
model = ILC(config)
model.B = torch.load(model_path / "B_10_179999.pt")
#model.B.data = torch.diag(torch.ones(config.n, dtype=config.dtype, device=config.device))
model.to(config.device)
model.eval()

eval_size = 2048
loss = torch.tensor(0, dtype=config.dtype, device=config.device)
z = torch.rand(eval_size, config.n, dtype=config.dtype, device=config.device)
for i in tqdm(range(eval_size)):
  y = calculate_y(config.n, z[i], model.B.data)
  loss += model(y)
print(loss.item() / eval_size)


<ipython-input-52-bcfc91f1769b>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.B = torch.load(model_path / "B_10_179999.pt")
100%|██████████| 2048/2048 [00:30<00:00,

0.07235777378082275
